In [1]:

import os
import json
import pandas as pd
import numpy as np
import pickle as pkl
import seaborn as sns
import gensim
from gensim.test.utils import common_texts
import torch

import torch.nn as nn
import torch.nn.functional as F
# load config
with open('config.json', 'r') as f:
    config = json.load(f)
cwd = os.getcwd()
os.chdir(config['REPODIR'])
import Utils as U
from Corpus import Corpus
os.chdir(cwd)

from collections import Counter, defaultdict
import itertools


from tqdm.auto import tqdm, trange
from collections import Counter
import random
from torch import optim
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

from torch.utils.tensorboard import SummaryWriter

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="dark")
import pickle as pkl
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
import nltk
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate

torch.set_default_dtype(torch.float32)

from torch.utils.data.dataloader import default_collate
from torch.utils.tensorboard import SummaryWriter
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
kwargs = {'num_workers': 4, 'pin_memory': True} \
         if (device == "cuda:0" or device == 'mps') else {}
collate_func = lambda x: tuple(x_.to(device) for x_ in default_collate(x)) \
               if device != "cpu" else default_collate

# with open('embedding_data.pkl', 'rb') as f:
#     embed = pkl.load(f)

# embed_df = pd.DataFrame(embed)


# n_classes = embed_df.author_id.nunique()

# from sklearn.preprocessing import OneHotEncoder
# label_encoder=OneHotEncoder(sparse_output=False)

# # -


# X = embed_df['vectors'] # Word Embeddings

# +
test_size = 0.2
val_size = 0.2
random_state =699


# +
# with open('embedding_data.pkl', 'rb') as f:
#     embed = pkl.load(f)

# embed_df = pd.DataFrame(embed)

data = U.load_file('data_vFFF.pkl', 'pkl', config['DATADIR'])


In [2]:

embed_df = pd.DataFrame(data)

embed_df['join_text'] = embed_df['text'].apply(' '.join)
embed_df = embed_df[['author_id', 'text', 'join_text', 'passage_key']]


In [3]:

n_classes = embed_df.author_id.nunique()

from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
y= label_encoder.fit_transform(embed_df['author_id'].to_numpy(dtype='int32').reshape(-1,1))
X = embed_df['passage_key']

X_train, X_test, y_train, y_test = U.train_test_split(X, y, test_size=test_size,
                                                        random_state=random_state,
                                                        stratify=y)

# Split train set into train and validation sets
X_train, X_val, y_train, y_val = U.train_test_split(X_train, y_train, test_size=val_size/(1-test_size),
                                                    random_state=random_state,
                                                    stratify=y_train)



/home/srtoner/.local/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [4]:
embed_df['label'] = y

train_set = set(X_train)
val_set = set(X_val)
test_set = set(X_test)

In [5]:
train = embed_df[embed_df['passage_key'].apply(lambda S: S in train_set)]
val = embed_df[embed_df['passage_key'].apply(lambda S: S in val_set)]
test = embed_df[embed_df['passage_key'].apply(lambda S: S in test_set)]

In [6]:
train.to_csv('train.csv', index=False)
val.to_csv('val.csv', index=False)
test.to_csv('test.csv', index=False)

In [7]:
train

,author_id,text,join_text,passage_key,label
1,2210,"[Bombay, for which they were now detained at C...","Bombay, for which they were now detained at Ca...",PG103_3686,104
3,2210,"[but the intractable Fogg, as reserved as ever...","but the intractable Fogg, as reserved as ever,...",PG103_3836,104
4,2210,"[“I am he.”, , “Is this man your servant?” add...",“I am he.” “Is this man your servant?” added ...,PG103_3536,104
5,2210,"[journey. But, if he thought of these possibil...","journey. But, if he thought of these possibili...",PG103_2186,104
6,2210,"[, “He is: a Frenchman, named Passepartout.”, ...","“He is: a Frenchman, named Passepartout.” “Y...",PG103_1886,104
...,...,...,...,...,...
99690,22568,"[appeared to him wrong and out of order, and t...","appeared to him wrong and out of order, and th...",PG53940_8141,237
99691,22568,"[the upper one, somewhat after the manner of t...","the upper one, somewhat after the manner of th...",PG53940_4091,237
99693,22568,"[And such a supper! Our host presided, and whi...","And such a supper! Our host presided, and whil...",PG53940_2441,237
99694,22568,"[himself, and you will lose him. Keep him in s...","himself, and you will lose him. Keep him in sl...",PG53940_4241,237


In [8]:
from datasets import Dataset

In [9]:
ds = {
      'train' :  Dataset.from_csv('train.csv'),
      'val' :  Dataset.from_csv('val.csv'),
      'test' :  Dataset.from_csv('test.csv'),
      }

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/srtoner/.cache/huggingface/datasets/csv/default-484655ef1c06ace4/0.0.0. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/srtoner/.cache/huggingface/datasets/csv/default-43d72da157862b30/0.0.0. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/srtoner/.cache/huggingface/datasets/csv/default-02f861fe847ad16a/0.0.0. Subsequent calls will reuse this data.


In [10]:
embed_df.columns

Index(['author_id', 'text', 'join_text', 'passage_key', 'label'], dtype='object')

In [11]:
train.to_csv('train.csv', index=False)
val.to_csv('val.csv', index=False)
test.to_csv('test.csv', index=False)

train

from datasets import Dataset

ds = {
      'train' :  Dataset.from_csv('train.csv'),
      'val' :  Dataset.from_csv('val.csv'),
      'test' :  Dataset.from_csv('test.csv'),
      }

embed_df.columns

type(embed_df.join_text.iloc[0])

str

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers.integrations import TensorBoardCallback
from transformers.trainer_callback import EarlyStoppingCallback
from torch.utils.data import DataLoader

BASE_MODEL = "allenai/longformer-base-4096"
# BASE_MODEL = "lreN5bs16" # Learning Rate 2e-5, batch size 16
LEARNING_RATE = 2e-5
MAX_LENGTH = 400
BATCH_SIZE = 4
EPOCHS = 3

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, 
                                                           num_labels=n_classes,
                                                           ignore_mismatched_sizes=True)

def preprocess_function(examples, test = False):
    # if not test:
    label = examples["label"] 
    examples = tokenizer(examples["text"],
                        truncation=True, 
                        padding="max_length",
                        max_length=MAX_LENGTH,
                        return_tensors='pt')
    
    for key in examples:
        examples[key] = examples[key].squeeze(0)
    # if not test:
    examples["label"] = torch.FloatTensor([label])
    examples = examples.to(device)
    return examples

for split in ds:
    ds[split] = ds[split].map(preprocess_function, 
                                remove_columns=['author_id', 'text', 'passage_key','join_text', 'label'])

    ds[split].set_format('pt')

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight'

Map:   0%|          | 0/59817 [00:00<?, ? examples/s]

Map:   0%|          | 0/19939 [00:00<?, ? examples/s]

Map:   0%|          | 0/19940 [00:00<?, ? examples/s]

In [13]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="output/",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    logging_steps = 1,
    num_train_epochs=.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1",
    greater_is_better=True,
    load_best_model_at_end=True,
    weight_decay=0.01,
    report_to = 'tensorboard'
)

early_stop = EarlyStoppingCallback(1, 0.01)
tb = TensorBoardCallback()


from transformers import Trainer
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss
    

In [14]:
from sklearn.metrics import f1_score

def compute_metrics_for_classification(eval_pred):
    predictions, labels = eval_pred
    labels = labels.reshape(-1, 1)
    f1 = f1_score(labels, predictions, squared=True)
    print(f"F1: {f1}")
    
    return {"F1": f1}

In [15]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["val"],
    compute_metrics=compute_metrics_for_classification,
    callbacks=[tb, early_stop]
)

You are adding a <class 'transformers.integrations.TensorBoardCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoardCallback


In [16]:
torch.cuda.memory_summary(device=device)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  582182 KB |  582182 KB |    1298 MB |  747720 KB |\n|       from large pool |  580870 KB |  580870 KB |     567 MB |       0 KB |\n|       from small pool |    1312 KB |    7500 KB |     731 MB |  747720 KB |\n|---------------------------------------------------------------------------|\n| Active memory         |  582182 KB |  582182 KB |    1298 MB |  747720 KB |\n|       from large pool |  580870 KB |  580870 KB |

In [17]:
trainer.train()

/home/srtoner/.local/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ValueError: Target size (torch.Size([4])) must be the same as input size (torch.Size([4, 241]))

In [ ]:
trainer.evaluate()

In [ ]:
print("Take 5")

prediction = trainer.predict(ds['validation'])

In [ ]:
trainer.save_model('hf_model')